In [1]:
from wandb.keras import WandbCallback
import wandb
from tqdm import tqdm
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
import numpy as np
import tensorflow as tf
import random
import os
import matplotlib.pyplot as plt

from utils import *

In [2]:
trainDataRaw = loadMidiInFolder(pathes['data_dir'], 'data/trainDataRaw')
testDataRaw = loadMidiInFolder(pathes['test_dir'], 'data/testDataRaw')

In [3]:
trainData = concatMidi(trainDataRaw)
testData = concatMidi(testDataRaw)

In [4]:
# Create dictionaries for each track or collect all (now).

beat2idx = []
idx2beat = []
params['vocab_size'] = []

vocabularies = set()

# Make dictionary across 4 tracks.
# for track in range(len(trainData)):
#     vocabularies = vocabularies | set(trainData[track]) | set(testData[track])
# for track in range(len(trainData)):
#     params['vocmakeSingleTrackModelab_size'].append(len(vocabularies))
#     beat2idx.append({beat: i for i, beat in enumerate(vocabularies)})
#     idx2beat.append({idx:beat for beat, idx in beat2idx[track].items()})

# Make dictionary for each track
for track in range(len(trainData)):
    vocabularies = set(trainData[track]) | set(testData[track])
    params['vocab_size'].append(len(vocabularies))
    beat2idx.append({beat: i for i, beat in enumerate(vocabularies)})
    idx2beat.append({idx:beat for beat, idx in beat2idx[track].items()})

In [5]:
params['vocab_size']

[296, 635, 813, 545]

In [6]:
for track in range(len(trainData)):

    trainData[track] = np.array([beat2idx[track][beat] for beat in trainData[track]])
    testData[track] = np.array([beat2idx[track][beat] for beat in testData[track]])

trainData = trainData.astype(int)
testData = testData.astype(int)

In [7]:
trainDatasets = []
testDatasets = []
for track in range(len(trainData)):
    trainDatasets.append(makeDataset(trainData[track], track))
    testDatasets.append(makeDataset(testData[track], track))

In [8]:
trainDatasets[0]

<BatchDataset shapes: ((64, 16), (64, 16, 296)), types: (tf.int64, tf.float32)>

## Single track training

In [9]:
name = 'single-track-lstm-training-wandb-exp-5'
track = 0
model = makeSingleTrackModel(track)
fig = plot_model(model, to_file=name+'.png', show_layer_names=True, show_shapes=True)

In [10]:
callbacks = []
filepath = config['path']['model_dir'] + \
    "model-"+name+"-{epoch:02d}-{loss:.4f}.hdf5"
callbacks.append(ModelCheckpoint(
    filepath=filepath,
    monitor='loss',
    mode='min',
    save_weights_only=True,
    save_best_only=True))

if outputs['wandb']:
    wandb.init(config=params, project='lstm-singletrack-js-fake')
    callbacks.append(WandbCallback(
        log_weights=True, log_evaluation=False, validation_steps=5))

history = model.fit(trainDatasets[track], validation_data=testDatasets[track], batch_size=params['batch_size'],
                    epochs=params['epochs'], verbose=1, callbacks=callbacks)

if outputs['wandb']:
    hist = np.histogram(trainData[track], bins=params['vocab_size'][track])
    topkWords = tf.math.top_k(hist[0], 10)
    topkChoicesWords = topkWords[1].numpy().squeeze()
    topkValuesWords = topkWords[0].numpy().squeeze()

    weight = model.get_weights()
    model2 = makeSingleTrackModel(track, 1)
    model2.set_weights(weight)

    model2.build(tf.TensorShape([1, None]))
    for index, idx in enumerate(topkChoicesWords):
        beat = idx2beat[track][idx]

        model2.reset_states()
        pred = model2(np.array([[idx]]))

        p = pred.numpy().squeeze()

        x_values = list(range(0, params['vocab_size'][track]))

        data = [[x, y] for (x, y) in zip(x_values, p)]
        table = wandb.Table(data=data, columns = ["idx", "prob"])
        wandb.log({f"softmax_distribution_top{index+1}" : wandb.plot.line(table, "idx", "prob",
                   title=f"Top {index+1} : {idx2beat[track][idx]} Softmax Prob Distribution")})

    for i in range(params['track_size']):
        x_values = list(range(0, params['vocab_size'][i]))
        hist = np.histogram(trainData[i], bins=params['vocab_size'][i])
        data = [[x, y] for (x, y) in zip(x_values, hist[0].tolist())]
        table = wandb.Table(data=data, columns = ["idx", "occurence"])
        wandb.log({f'word_distribution_track_{i}' : wandb.plot.line(table, "idx", "occurence",
                   title=f"Track {i} Word distribution.")})
    
    wandb.log({'model_architecture': wandb.Image(name+'.png')})
    wandb.finish()


wandb: Currently logged in as: stu00608 (tku-cilab). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/500
21/21 [==============================] - 4s 69ms/step - loss: 5.6482 - acc: 0.0382 - val_loss: 5.6689 - val_acc: 0.1335
Epoch 2/500
21/21 [==============================] - 1s 43ms/step - loss: 5.3082 - acc: 0.1189 - val_loss: 5.6235 - val_acc: 0.1341
Epoch 3/500
21/21 [==============================] - 1s 45ms/step - loss: 4.7934 - acc: 0.1286 - val_loss: 5.6103 - val_acc: 0.1364
Epoch 4/500
21/21 [==============================] - 1s 41ms/step - loss: 4.4993 - acc: 0.1379 - val_loss: 5.5787 - val_acc: 0.1403
Epoch 5/500
21/21 [==============================] - 1s 39ms/step - loss: 4.2331 - acc: 0.1410 - val_loss: 5.5285 - val_acc: 0.1453
Epoch 6/500
21/21 [==============================] - 1s 39ms/step - loss: 4.0527 - acc: 0.1494 - val_loss: 5.4731 - val_acc: 0.1470
Epoch 7/500
21/21 [==============================] - 1s 47ms/step - loss: 3.9232 - acc: 0.1463 - val_loss: 5.4120 - val_acc: 0.1483
Epoch 8/500
21/21 [==============================] - 1s 49ms/step - loss: 3.

21/21 [==============================] - 1s 38ms/step - loss: 2.3764 - acc: 0.3061 - val_loss: 2.5696 - val_acc: 0.2860
Epoch 63/500
21/21 [==============================] - 1s 37ms/step - loss: 2.3786 - acc: 0.3117 - val_loss: 2.5679 - val_acc: 0.2803
Epoch 64/500
21/21 [==============================] - 1s 37ms/step - loss: 2.3650 - acc: 0.3139 - val_loss: 2.5814 - val_acc: 0.2808
Epoch 65/500
21/21 [==============================] - 1s 37ms/step - loss: 2.3655 - acc: 0.3123 - val_loss: 2.5629 - val_acc: 0.2829
Epoch 66/500
21/21 [==============================] - 1s 37ms/step - loss: 2.3397 - acc: 0.3216 - val_loss: 2.5561 - val_acc: 0.2886
Epoch 67/500
21/21 [==============================] - 1s 37ms/step - loss: 2.3343 - acc: 0.3216 - val_loss: 2.5564 - val_acc: 0.2799
Epoch 68/500
21/21 [==============================] - 1s 41ms/step - loss: 2.3341 - acc: 0.3160 - val_loss: 2.5538 - val_acc: 0.2907
Epoch 69/500
21/21 [==============================] - 1s 37ms/step - loss: 2.3236 

21/21 [==============================] - 1s 37ms/step - loss: 1.9388 - acc: 0.3953 - val_loss: 2.5999 - val_acc: 0.2817
Epoch 124/500
21/21 [==============================] - 1s 37ms/step - loss: 1.9476 - acc: 0.3957 - val_loss: 2.5943 - val_acc: 0.2764
Epoch 125/500
21/21 [==============================] - 1s 37ms/step - loss: 1.9315 - acc: 0.3993 - val_loss: 2.6033 - val_acc: 0.2770
Epoch 126/500
21/21 [==============================] - 1s 37ms/step - loss: 1.9038 - acc: 0.4067 - val_loss: 2.5987 - val_acc: 0.2729
Epoch 127/500
21/21 [==============================] - 1s 38ms/step - loss: 1.9103 - acc: 0.4106 - val_loss: 2.5926 - val_acc: 0.2743
Epoch 128/500
21/21 [==============================] - 1s 37ms/step - loss: 1.9040 - acc: 0.4036 - val_loss: 2.6075 - val_acc: 0.2762
Epoch 129/500
21/21 [==============================] - 1s 37ms/step - loss: 1.8912 - acc: 0.4061 - val_loss: 2.6233 - val_acc: 0.2786
Epoch 130/500
21/21 [==============================] - 1s 37ms/step - loss: 

...

21/21 [==============================] - 1s 38ms/step - loss: 0.9100 - acc: 0.7101 - val_loss: 3.3162 - val_acc: 0.2588
Epoch 307/500
21/21 [==============================] - 1s 37ms/step - loss: 0.9243 - acc: 0.7087 - val_loss: 3.3864 - val_acc: 0.2550
Epoch 308/500
21/21 [==============================] - 1s 37ms/step - loss: 0.8920 - acc: 0.7200 - val_loss: 3.3923 - val_acc: 0.2580
Epoch 309/500
21/21 [==============================] - 1s 37ms/step - loss: 0.8939 - acc: 0.7183 - val_loss: 3.4073 - val_acc: 0.2575
Epoch 310/500
21/21 [==============================] - 1s 37ms/step - loss: 0.8993 - acc: 0.7150 - val_loss: 3.4066 - val_acc: 0.2555
Epoch 311/500
21/21 [==============================] - 1s 37ms/step - loss: 0.8944 - acc: 0.7177 - val_loss: 3.4206 - val_acc: 0.2520
Epoch 312/500
21/21 [==============================] - 1s 37ms/step - loss: 0.8880 - acc: 0.7190 - val_loss: 3.4031 - val_acc: 0.2580
Epoch 313/500
21/21 [==============================] - 1s 41ms/step - loss: 

21/21 [==============================] - 1s 38ms/step - loss: 0.7028 - acc: 0.7802 - val_loss: 3.7184 - val_acc: 0.2505
Epoch 368/500
21/21 [==============================] - 1s 38ms/step - loss: 0.7110 - acc: 0.7747 - val_loss: 3.6953 - val_acc: 0.2508
Epoch 369/500
21/21 [==============================] - 1s 38ms/step - loss: 0.7007 - acc: 0.7770 - val_loss: 3.6932 - val_acc: 0.2539
Epoch 370/500
21/21 [==============================] - 1s 38ms/step - loss: 0.7004 - acc: 0.7773 - val_loss: 3.6774 - val_acc: 0.2557
Epoch 371/500
21/21 [==============================] - 1s 38ms/step - loss: 0.6956 - acc: 0.7788 - val_loss: 3.6968 - val_acc: 0.2498
Epoch 372/500
21/21 [==============================] - 1s 38ms/step - loss: 0.6968 - acc: 0.7808 - val_loss: 3.7259 - val_acc: 0.2524
Epoch 373/500
21/21 [==============================] - 1s 38ms/step - loss: 0.6910 - acc: 0.7846 - val_loss: 3.6657 - val_acc: 0.2591
Epoch 374/500
21/21 [==============================] - 1s 38ms/step - loss: 

21/21 [==============================] - 1s 46ms/step - loss: 0.5693 - acc: 0.8205 - val_loss: 4.0107 - val_acc: 0.2482
Epoch 429/500
21/21 [==============================] - 1s 42ms/step - loss: 0.5617 - acc: 0.8220 - val_loss: 3.9991 - val_acc: 0.2472
Epoch 430/500
21/21 [==============================] - 1s 37ms/step - loss: 0.5650 - acc: 0.8210 - val_loss: 4.0192 - val_acc: 0.2448
Epoch 431/500
21/21 [==============================] - 1s 40ms/step - loss: 0.5637 - acc: 0.8174 - val_loss: 4.0337 - val_acc: 0.2507
Epoch 432/500
21/21 [==============================] - 1s 37ms/step - loss: 0.5599 - acc: 0.8212 - val_loss: 3.9867 - val_acc: 0.2555
Epoch 433/500
21/21 [==============================] - 1s 38ms/step - loss: 0.5623 - acc: 0.8203 - val_loss: 4.0585 - val_acc: 0.2453
Epoch 434/500
21/21 [==============================] - 1s 43ms/step - loss: 0.5529 - acc: 0.8232 - val_loss: 4.0189 - val_acc: 0.2474
Epoch 435/500
21/21 [==============================] - 1s 45ms/step - loss: 

21/21 [==============================] - 1s 37ms/step - loss: 0.4742 - acc: 0.8478 - val_loss: 4.3101 - val_acc: 0.2443
Epoch 490/500
21/21 [==============================] - 1s 37ms/step - loss: 0.4745 - acc: 0.8481 - val_loss: 4.2743 - val_acc: 0.2484
Epoch 491/500
21/21 [==============================] - 1s 37ms/step - loss: 0.4703 - acc: 0.8490 - val_loss: 4.2933 - val_acc: 0.2427
Epoch 492/500
21/21 [==============================] - 1s 37ms/step - loss: 0.4695 - acc: 0.8495 - val_loss: 4.3047 - val_acc: 0.2437
Epoch 493/500
21/21 [==============================] - 1s 40ms/step - loss: 0.4753 - acc: 0.8455 - val_loss: 4.3095 - val_acc: 0.2437
Epoch 494/500
21/21 [==============================] - 1s 37ms/step - loss: 0.4685 - acc: 0.8491 - val_loss: 4.3306 - val_acc: 0.2422
Epoch 495/500
21/21 [==============================] - 1s 37ms/step - loss: 0.4618 - acc: 0.8496 - val_loss: 4.3531 - val_acc: 0.2414
Epoch 496/500
21/21 [==============================] - 1s 37ms/step - loss: 

acc,▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▆▇███▇██▇▇▇▇▇▇▇▇▆▇▇▇▇▆▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
val_loss,█▆▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅
acc,0.84882
best_epoch,84
best_val_loss,2.50486
epoch,499
loss,0.4657
val_acc,0.24447


In [12]:
params['vocab_size']

[296, 635, 813, 545]

## Multi track training

In [ ]:
x_train = []
y_train = []
x_val = []
y_val = []

for track in range(params['track_size']):

    x = list(trainDatasets[track].map(lambda x, y: x))[0].numpy()
    y = list(trainDatasets[track].map(lambda x, y: y))[0].numpy()

    val_x = list(testDatasets[track].map(lambda x, y: x))[0].numpy()
    val_y = list(testDatasets[track].map(lambda x, y: y))[0].numpy()

    x_train.append(x)
    y_train.append(y)
    x_val.append(val_x)
    y_val.append(val_y)

In [ ]:
x_train[0].shape, y_train[0].shape, x_val[0].shape, y_val[0].shape

In [ ]:
name = 'multi-track-lstm-training-5'
model = makeMultiTrackModel()

In [ ]:
callbacks = []
filepath = config['path']['model_dir'] + \
    "model-"+name+"-{epoch:02d}-{loss:.4f}.hdf5"
callbacks.append(ModelCheckpoint(
    filepath=filepath,
    monitor='loss',
    mode='min',
    save_weights_only=True,
    save_best_only=True))

if outputs['wandb']:
    wandb.init(config=params, project='lstm-multi-track-js-fake')
    callbacks.append(WandbCallback(
        log_weights=True, log_evaluation=False, validation_steps=5))

history = model.fit(x=x_train, y=y_train, validation_data=(x_val, y_val), batch_size=params['batch_size'],
                    epochs=params['epochs'], verbose=1, callbacks=callbacks)

if outputs['wandb']:
    
    wandb.finish()

## Generate Single Track

In [ ]:
initBeatIndex = random.randint(0, len(trainData[0])-params['sequence_length'])
initBeat = np.array([trainData[0, initBeatIndex:initBeatIndex+params['sequence_length']]])

In [ ]:
initBeat.shape

In [ ]:
weight = 'model-single-track-lstm-training-05-4.0770.hdf5'

model = makeSingleTrackModel(track, 1)
model.build(tf.TensorShape([1, None]))
model.load_weights(os.path.join(pathes['model_dir'], weight))
model.reset_states()
pred = model(initBeat)
pred = pred.numpy().squeeze()

music_sequence = []

for i in tqdm(range(outputs['length'])):

    topk = tf.math.top_k(pred[-1], 3)
    topkChoices = topk[1].numpy().squeeze()
    topkValues = topk[0].numpy().squeeze()

    # Apply random
    next_beat = None
    if np.random.uniform(0, 1) < .5:
        next_beat = topkChoices[0]
    else:
        p_choices = tf.math.softmax(topkValues[1:]).numpy()
        next_beat = np.random.choice(topkChoices[1:], 1, p=p_choices)[0]

    music_sequence.append(idx2beat[track][next_beat])

    pred = model(np.array([[next_beat]]))
    pred = tf.expand_dims(pred, 0)

In [ ]:
generateMidi(name+'.mid', [music_sequence])

## Generate Multi Track

In [ ]:
initBeatIndex = random.randint(0, len(trainData[0])-params['sequence_length'])
initBeat = []
for track in range(params['track_size']):
    initBeat.append(np.array([trainData[track, initBeatIndex:initBeatIndex+params['sequence_length']]]))

In [ ]:
weight = 'model-multi-track-lstm-training-4-998-1.0914.hdf5'

In [ ]:
model = makeMultiTrackModel(1)
model.build(tf.TensorShape([1, None]))
model.load_weights(os.path.join(pathes['model_dir'], weight))
model.reset_states()
pred = model(initBeat)
pred = [p.numpy().squeeze() for p in pred]

music_sequence = []
for track in range(params['track_size']):
    music_sequence.append([idx2beat[track][idx] for idx in initBeat[track][0].tolist()])

outputs['length'] = 100
for i in tqdm(range(outputs['length'])):

    nextBeatList = []
    for index, p in enumerate(pred):
        topk = tf.math.top_k(p[-1], 3)
        topkChoices = topk[1].numpy().squeeze()
        topkValues = topk[0].numpy().squeeze()

        # Apply random
        # next_beat = topkChoices[0]
        next_beat = None
        if np.random.uniform(0, 1) < .5:
            next_beat = topkChoices[0]
        else:
            p_choices = tf.math.softmax(topkValues[1:]).numpy()
            next_beat = np.random.choice(topkChoices[1:], 1, p=p_choices)[0]

        music_sequence[index].append(idx2beat[index][next_beat])
        nextBeatList.append(np.array([[next_beat]]))

    pred = model.predict(nextBeatList, verbose=0)


In [ ]:
generateMidi('new100-4-2'+'.mid', music_sequence)

## Output

In [ ]:
fig = plot_model(model, to_file=name+'.png', show_shapes=True, show_layer_names=True)